In [28]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
import json
from tqdm import tqdm

load_dotenv()

def get_links(url):
    # Enviando requisição HTTP para obter o conteúdo da página
    response = requests.get(url)
    content = response.content

    # Analisando o conteúdo da página com BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Encontrar todas as tags ul que tenham archive-list__list na classe
    archive_list = soup.find('ul', class_='archive-list__list')

    # Encontrar todas as tags <li> dentro da tag <ul>
    li_tags = archive_list.find_all('li')

    links = []
    # Iterar sobre todas as tags <li> e extrair o href
    for li in li_tags:
        a_tag = li.find('a')
        href = a_tag['href']
        links.append(href)
    return links


In [29]:
def extract_article_info(url):
    try:
        # Fazer a requisição HTTP
        response = requests.get(url)
        
        # Verificar se a requisição foi bem-sucedida
        if response.status_code != 200:
            return f"Erro ao acessar a página: {response.status_code}"
            
        # Definir a codificação como UTF-8
        response.encoding = 'utf-8'

        # Fazer o parsing do HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extrair informações do artigo
        if url.startswith('https://www.poder360.com.br/poder-flash'):
            title = soup.find('h1', class_='box-poder-flash__title mt-4').text.strip()
        else:
            title = soup.find('h1', class_='inner-page-section__title title-1').text.strip()
        subtitle = soup.find('h2', class_='inner-page-section__line').text.strip()
        
        # Tentar encontrar a data e o autor
        date = soup.find('time')  # Assumindo que a data está dentro de uma tag <time>
        if date:
            date = date.text.strip()
        else:
            date = "Data não encontrada"
        
        # Extrair o autor
        author_div = soup.find('div', class_='footer-post__box')
        if author_div:
            author_name = author_div.find('a', class_='author__name')
            if author_name:
                author = author_name.text.strip()
            else:
                author = "Autor não encontrado"
        else:
            author = "Autor não encontrado"
            
        label = url.replace('https://www.poder360.com.br/', '').split('/')[0]
        
        # Extrair o texto do artigo
        article_body = soup.find('div', class_='inner-page-section__text')
        if article_body:
            # Encontra e remove o elemento <form> dentro de article_body se ele existir
            form = article_body.find('form')
            if form:
                form.decompose()

            # Inicializa uma lista para armazenar o texto dos elementos
            text_elements = []

            # Itera sobre todos os filhos do article_body
            for child in article_body.children:
                if child.name == 'p':
                    text_elements.append(child.get_text(strip=True))
                elif child.name == 'ul':
                    # Itera sobre todos os <li> dentro do <ul>
                    for li in child.find_all('li'):
                        text_elements.append(li.get_text(strip=True))

            # Combina todos os textos em uma única string separada por quebras de linha
            text = '\n'.join(text_elements)
        else:
            text = "Texto do artigo não encontrado"
        
        metadados = {
            "titulo": title,
            "subtitulo": subtitle,
            "data": date,
            "autor": author,
            "texto": text,
            "label": label,
            "url": url
        }
        
        return metadados
    except Exception as e:
        print(f"Erro ao extrair informações do artigo: {e}")
        print(f"URL do artigo: {url}")
        


In [30]:
def get_content(n_dias, save_local=False):
    all_data = []
    # Calcular a data de ontem
    for i in tqdm(range(1, n_dias)):
        yesterday = datetime.now() - timedelta(i)
        yesterday_str = yesterday.strftime('%Y-%m-%d')
        filename = f'../data/news_{yesterday_str}.json'
        if os.path.exists(filename):
            print(f"File {filename} already exists")
            continue
        #construir a url das notícias de ontem
        url = f'https://www.poder360.com.br/{yesterday.strftime("%Y/%m/%d")}'
        print(f"Date link: {url}")

        links = get_links(url)
        links_filtrados = [link for link in links if not link.startswith("https://www.poder360.com.br/author/")]
        print(f"Cron job links: {len(links_filtrados)}")

        news = []
        for link in links_filtrados:
            metadado = extract_article_info(link)
            news.append(metadado)
        
        if save_local:
            # Verificar se o diretório 'data' existe, se não, criar
            if not os.path.exists('../data'):
                os.makedirs('../data')
            # Nome do arquivo
            

            # Tentar abrir o arquivo e ler seu conteúdo, se existir
            try:
                with open(filename, 'r', encoding='utf-8') as f:
                    data = json.load(f)
            except FileNotFoundError:
                data = []

            # Adicionar a nova notícia à lista
            data.append(news)

            # Salvar a lista de notícias no arquivo JSON
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            all_data.append(news)
        print("Cron job finished")
    return all_data

In [31]:
response = get_content(90, save_local=True)

  0%|          | 0/89 [00:00<?, ?it/s]

File ../data/news_2024-11-02.json already exists
File ../data/news_2024-11-01.json already exists
File ../data/news_2024-10-31.json already exists
File ../data/news_2024-10-30.json already exists
File ../data/news_2024-10-29.json already exists
File ../data/news_2024-10-28.json already exists
File ../data/news_2024-10-27.json already exists
File ../data/news_2024-10-26.json already exists
File ../data/news_2024-10-25.json already exists
File ../data/news_2024-10-24.json already exists
File ../data/news_2024-10-23.json already exists
File ../data/news_2024-10-22.json already exists
File ../data/news_2024-10-21.json already exists
File ../data/news_2024-10-20.json already exists
File ../data/news_2024-10-19.json already exists
File ../data/news_2024-10-18.json already exists
File ../data/news_2024-10-17.json already exists
File ../data/news_2024-10-16.json already exists
File ../data/news_2024-10-15.json already exists
File ../data/news_2024-10-14.json already exists
File ../data/news_20

 90%|████████▉ | 80/89 [03:36<00:24,  2.71s/it]

Cron job finished
Date link: https://www.poder360.com.br/2024/08/14
Cron job links: 132


 91%|█████████ | 81/89 [07:51<00:57,  7.13s/it]

...

Cron job finished
Date link: https://www.poder360.com.br/2024/08/07
Cron job links: 116


 99%|█████████▉| 88/89 [31:38<01:25, 85.20s/it]

Cron job finished
Date link: https://www.poder360.com.br/2024/08/06
Cron job links: 107


100%|██████████| 89/89 [35:01<00:00, 23.61s/it] 

Cron job finished


In [13]:
import pandas as pd 
import os
from pprint import pprint

# Load data
paths =[x for x in os.listdir('../data') if x.endswith('.json')]

dfs = []
for path in paths:
    with open(f'../data/{path}', encoding='utf8') as f:
        data = json.load(f)[0]
        try:
            dfs.append(pd.DataFrame(data))
        except:
            data_ = []
            for idx , x in enumerate(data):
                if x is None:
                    print(f"None in {idx}")
                else:
                    data_.append(x)
            dfs.append(pd.DataFrame(data_))
df = pd.concat(dfs).reset_index(drop=True)

None in 39
None in 81
None in 29


In [14]:
df

,titulo,subtitulo,data,autor,texto,label,url
0,Governo espera R$ 166 bilhões para aumentar a ...,Números constam no Projeto de Lei Orçamentária...,30.ago.2024 (sexta-feira) - 23h55,Gabriel Benevides redator,O governo federal espera que as medidas ligada...,poder-economia,https://www.poder360.com.br/poder-economia/gov...
1,"XP foi muito generosa com o cachê, diz Abel Fe...",Técnico do Palmeiras participou na noite desta...,30.ago.2024 (sexta-feira) - 23h48,Victor Boscato estagiário,"O técnico do Palmeiras, Abel Ferreira, disse q...",poder-sportsmkt,https://www.poder360.com.br/poder-sportsmkt/xp...
2,Musk incentiva o uso de VPN para acesso ao X n...,"Mais cedo, o ministro Alexandre de Moraes mand...",30.ago.2024 (sexta-feira) - 23h15,PODER360,"O dono doX(ex-Twitter), Elon Musk, incentivou ...",poder-justica,https://www.poder360.com.br/poder-justica/musk...
3,"China, Rússia e Coreia do Norte: saiba onde o ...","Ao todo, 6 países restringem o acesso à plataf...",30.ago.2024 (sexta-feira) - 23h09,PODER360,O ministro do STF (Supremo Tribunal Federal) A...,poder-internacional,https://www.poder360.com.br/poder-internaciona...
4,Empreendedorismo é o ministério com menor orça...,"Órgão terá R$ 132,6 milhões disponíveis no ano...",30.ago.2024 (sexta-feira) - 23h02,Vitória Queiroz repórter,O Ministério do Empreendedorismo é oministério...,poder-empreendedor,https://www.poder360.com.br/poder-empreendedor...
...,...,...,...,...,...,...,...
6212,"Com incerteza fiscal, dólar segue acima de R$ ...","A moeda norte-americana fechou cotada a R$ 5,8...",2.nov.2024 (sábado) - 5h55,Houldine Nascimento repórter,A cotação do dólar está acima de R$ 5 há 219 d...,poder-economia,https://www.poder360.com.br/poder-economia/com...
6213,Leia a projeção de delegados para Kamala e Tru...,São necessários ao menos 270 para ser eleito; ...,2.nov.2024 (sábado) - 5h50,PODER360,A 3 dias da eleição presidencial nos Estados U...,poder-internacional,https://www.poder360.com.br/poder-internaciona...
6214,TCE-RJ suspende pagamentos a laboratório de ca...,PCS Saleme foi responsável pelo exame de sangu...,2.nov.2024 (sábado) - 5h45,PODER360,O TCE-RJ (Tribunal de Contas do Estado do Rio ...,poder-justica,https://www.poder360.com.br/poder-justica/tce-...
6215,Brasil inicia novembro com 1.746 focos de incê...,Dados são de 6ª feira (1º.nov.2024); Amazônia ...,2.nov.2024 (sábado) - 4h49,PODER360,O Brasilregistrava1.746de incêndio na 6ª feira...,poder-sustentavel,https://www.poder360.com.br/poder-sustentavel/...


In [18]:
# Add quebra de linha '\n'
all_data = df.copy()
all_data['text'] = df['titulo'] + '\n' + df['subtitulo'] + '\n' + df['texto']
all_data['class'] = df['label']
all_data['url'] = df['url']
all_data = all_data[['text', 'class', 'url']]
label_counts = all_data['class'].value_counts()
labels_above_100 = label_counts[label_counts > 400].index
data = all_data[all_data['class'].isin(labels_above_100)]

In [19]:
data['class'].value_counts()

class
poder-eleicoes         1128
poder-internacional     798
poder-economia          644
poder-justica           533
poder-governo           527
poder-sportsmkt         404
poder-congresso         401
Name: count, dtype: int64

In [20]:
data

,text,class,url
0,Governo espera R$ 166 bilhões para aumentar a ...,poder-economia,https://www.poder360.com.br/poder-economia/gov...
1,"XP foi muito generosa com o cachê, diz Abel Fe...",poder-sportsmkt,https://www.poder360.com.br/poder-sportsmkt/xp...
2,Musk incentiva o uso de VPN para acesso ao X n...,poder-justica,https://www.poder360.com.br/poder-justica/musk...
3,"China, Rússia e Coreia do Norte: saiba onde o ...",poder-internacional,https://www.poder360.com.br/poder-internaciona...
5,"Governo propõe verba de R$ 60,9 bilhões ao PAC...",poder-economia,https://www.poder360.com.br/poder-economia/gov...
...,...,...,...
6211,Real é a 7ª moeda que mais se desvalorizou no ...,poder-economia,https://www.poder360.com.br/poder-economia/rea...
6212,"Com incerteza fiscal, dólar segue acima de R$ ...",poder-economia,https://www.poder360.com.br/poder-economia/com...
6213,Leia a projeção de delegados para Kamala e Tru...,poder-internacional,https://www.poder360.com.br/poder-internaciona...
6214,TCE-RJ suspende pagamentos a laboratório de ca...,poder-justica,https://www.poder360.com.br/poder-justica/tce-...
